In [1]:
pip install refractio[snowflake]


     |████████████████████████████████| 12.3MB 5.0MB/s eta 0:00:01
     |████████████████████████████████| 2.5MB 82.1MB/s eta 0:00:01
     |████████████████████████████████| 512kB 76.6MB/s eta 0:00:01
     |████████████████████████████████| 17.3MB 74.7MB/s eta 0:00:01
     |████████████████████████████████| 348kB 77.0MB/s eta 0:00:01
     |████████████████████████████████| 235kB 74.7MB/s eta 0:00:01
     |████████████████████████████████| 61kB 20.6MB/s eta 0:00:01
     |████████████████████████████████| 174kB 83.4MB/s eta 0:00:01
     |████████████████████████████████| 143kB 75.6MB/s eta 0:00:01
     |████████████████████████████████| 4.4MB 77.6MB/s eta 0:00:01
     |████████████████████████████████| 71kB 21.5MB/s eta 0:00:01
     |████████████████████████████████| 153kB 77.2MB/s eta 0:00:01
     |████████████████████████████████| 61kB 19.9MB/s eta 0:00:01
     |████████████████████████████████| 71kB 21.3MB/s eta 0:00:01
     |████████████████████████████████| 450kB 81.1MB/s eta 0:00:0

In [2]:
from refractio import snowflake
dataset = snowflake.execute_query(f" SELECT * FROM RETAIL_DB.RETAIL_MKT_BSKT_SCHEMA.ORDER_INVENTORY")
dataset

User name picked from OS env: vikaskumar.10710602@ltimindtree.com
Fetching connections created by vikaskumar.10710602@ltimindtree.com user
Connection names fetched ['RETAIL_MKT_BSKT_CXN'], created by vikaskumar.10710602@ltimindtree.com
Connection object created: <snowflake.connector.connection.SnowflakeConnection object at 0x7f70f43be1f0>
Please close the connection after use!


,ORDER_DATE,ORDER_ID,PRODUCT_ID,PRODUCT,QUANTITY,UNIT_PRICE,SHIP_MODE,TOTAL_SALES,CUSTOMER_ID
0,1/31/2021 0:00,BU-2013-7380,10002439,shrimp,167,74.69,Standard Class,12473.23,MH-7455
1,1/31/2021 0:00,BU-2013-7380,10000353,almonds,278,15.28,Standard Class,4247.84,MH-7455
2,1/31/2021 0:00,BU-2013-7380,10000994,avocado,324,46.33,Standard Class,15010.92,MH-7455
3,1/31/2021 0:00,BU-2013-7380,10000475,vegetables mix,124,58.22,Standard Class,7219.28,MH-7455
4,1/31/2021 0:00,BU-2013-7380,10003338,green grapes,680,86.31,Standard Class,58690.80,MH-7455
...,...,...,...,...,...,...,...,...,...
72624,3/30/2023 0:00,US-2013-151225,10004883,chicken,290,86.72,Standard Class,25148.80,GK-14620
72625,3/30/2023 0:00,US-2013-151225,10001990,tropical fruit,314,56.47,Standard Class,17731.58,GK-14620
72626,3/30/2023 0:00,US-2013-151225,10000425,other vegetables,447,92.13,Standard Class,41182.11,GK-14620
72627,3/30/2023 0:00,US-2013-151225,10001410,vinegar,701,89.69,Standard Class,62872.69,GK-14620


In [4]:
import pandas as pd
df = pd.DataFrame(dataset)
product_set=df.groupby("ORDER_ID")["PRODUCT"].apply(set).to_dict()
len(product_set)

15277

In [1]:
filter_basket = {key:value for key, value in product_set.items() if len(value)>1}
#print (filter_basket)
#final_basket=pd.DataFrame(filter_basket)
#final_basket
List_order_id=list(filter_basket.keys())
#print (List_order_id)
List_product_id=[list(values) for values in filter_basket.values()]
#print (List_product_id)
#final_basket
dict1={"ORDER_ID":List_order_id,"Product_ID":List_product_id}
#print(dict1)
df1=pd.DataFrame(dict1)
print(df1)

Package                    Version    
-------------------------- -----------
aiohttp                    3.9.3      
aiosignal                  1.3.1      
alembic                    1.13.1     
anyio                      4.2.0      
archspec                   0.2.1      
argon2-cffi                23.1.0     
argon2-cffi-bindings       21.2.0     
asn1crypto                 1.5.1      
asttokens                  2.4.1      
async-generator            1.10       
async-timeout              4.0.3      
attrs                      23.2.0     
Babel                      2.14.0     
beautifulsoup4             4.12.3     
bleach                     4.1.0      
boltons                    23.0.0     
bqplot                     0.12.42    
Brotli                     1.0.9      
certifi                    2024.6.2   
certipy                    0.1.3      
cffi                       1.16.0     
charset-normalizer         3.3.2      
click                      8.1.7      
clickclick               

In [6]:
pip uninstall numpy -y

Uninstalling numpy-2.0.0:
  Successfully uninstalled numpy-2.0.0
Note: you may need to restart the kernel to use updated packages.
